# 1.6 Agregaciones y KPIs

- Objetivo: calcular métricas agregadas (SUM, AVG, COUNT) y KPIs de negocio.
- Prerrequisitos: entender GROUP BY, funciones agregadas, diferencia entre COUNT(*) y COUNT(DISTINCT).
- Ejercicios: ingreso bruto, margen, ticket promedio, top productos.
- Reto: KPIs complejos con múltiples niveles de agregación.
- Errores comunes: olvidar DISTINCT en conteos, mezclar granularidades.

## ¿Por qué calcular KPIs con SQL?

Los **KPIs (Key Performance Indicators)** son métricas que resumen el desempeño del negocio. SQL es la herramienta ideal para calcularlos porque:

- Procesa millones de registros eficientemente
- Centraliza lógica de negocio (todos usan la misma definición de "ticket promedio")
- Alimenta dashboards en tiempo real (Power BI, Tableau, Looker)

**Para qué:**
- Monitorear salud del negocio (ingresos diarios, margen, tasa de conversión)
- Comparar períodos (mes actual vs mes anterior)
- Identificar outliers (días con ventas anormalmente bajas)

**Cómo:**
- **SUM:** Total acumulado (ingresos totales, unidades vendidas)
- **AVG:** Promedio (ticket promedio, descuento promedio)
- **COUNT:** Conteo (número de transacciones, clientes únicos)
- **MIN/MAX:** Extremos (venta mínima, fecha más reciente)

## Funciones de Agregación

### SUM - Suma Total
```sql
SELECT SUM(cantidad) AS unidades_totales
FROM fact_ventas;
```
**Uso:** Ingresos totales, unidades vendidas, costo total de inventario.

### AVG - Promedio
```sql
SELECT AVG(precio_unitario) AS precio_promedio
FROM dim_productos;
```
**Cuidado:** AVG(cantidad*precio) NO es lo mismo que SUM(cantidad*precio) / COUNT(*). Usar subconsultas cuando sea necesario.

### COUNT - Conteo
```sql
-- Transacciones totales
SELECT COUNT(*) FROM fact_ventas;

-- Clientes únicos que compraron
SELECT COUNT(DISTINCT cliente_id) FROM fact_ventas;
```
**COUNT(*):** Todas las filas (incluyendo duplicados)
**COUNT(DISTINCT col):** Valores únicos de columna (elimina duplicados)
**COUNT(col):** Cuenta solo valores no NULL

### GROUP BY - Agrupar por Dimensión
```sql
SELECT fecha, SUM(cantidad) AS unidades_dia
FROM fact_ventas
GROUP BY fecha;
```
**Regla:** Toda columna en SELECT que no esté en función agregada DEBE estar en GROUP BY.

## KPIs de Negocio Comunes

### Ticket Promedio
**Definición:** Ingreso promedio por transacción

```sql
SELECT AVG(cantidad * precio_unitario * (1 - descuento_pct/100)) AS ticket_promedio
FROM fact_ventas v
JOIN dim_productos p ON v.producto_id = p.producto_id;
```

### Margen Bruto
**Definición:** Ingresos - Costos

```sql
SELECT 
  SUM(cantidad * precio_unitario * (1 - descuento_pct/100)) AS ingresos,
  SUM(cantidad * costo_unitario) AS costos,
  SUM(cantidad * precio_unitario * (1 - descuento_pct/100)) - SUM(cantidad * costo_unitario) AS margen_bruto
FROM fact_ventas v
JOIN dim_productos p ON v.producto_id = p.producto_id;
```

### Tasa de Conversión (Clientes que compraron / Total visitantes)
*Requiere tabla de visitas*

### Churn Rate (Clientes que dejaron de comprar)
*Análisis de cohortes, tema avanzado*

## Errores Comunes

1. **Olvidar DISTINCT en conteos de clientes únicos:**
   ```sql
   -- INCORRECTO: cuenta transacciones, no clientes
   SELECT COUNT(cliente_id) FROM fact_ventas;
   
   -- CORRECTO: clientes únicos
   SELECT COUNT(DISTINCT cliente_id) FROM fact_ventas;
   ```

2. **Mezclar granularidades:**
   ```sql
   -- INCORRECTO: margen_pct debe calcularse DESPUÉS de sumar
   SELECT AVG(margen_pct) FROM productos;  -- Promedio de promedios (mal)
   
   -- CORRECTO: sumar ingresos y costos, luego dividir
   SELECT SUM(ingresos) / SUM(costos) AS margen_pct;
   ```

3. **No separar costos de ingresos:**
   Calcular margen como `SUM(precio - costo)` puede ocultar problemas (alto ingreso con alto costo vs bajo ingreso con bajo costo).

4. **Promediar promedios:**
   Si tienes ingreso promedio diario, el promedio mensual NO es `AVG(ingreso_promedio_dia)` sino `SUM(ingresos_mes) / días_mes`.

In [ ]:
-- Unidades totales y clientes distintos por día
SELECT fecha, SUM(cantidad) AS unidades_totales, COUNT(DISTINCT cliente_id) AS clientes_distintos
FROM dbo.fact_ventas
GROUP BY fecha
ORDER BY fecha;

In [ ]:
-- Ticket promedio (ingreso promedio por venta)
SELECT AVG(v.cantidad * p.precio_unitario * (1 - v.descuento_pct/100)) AS ticket_promedio
FROM dbo.fact_ventas v
JOIN dbo.dim_productos p ON v.producto_id = p.producto_id;

🟢 Ejercicio: Ingreso bruto por día y su promedio diario global.

In [ ]:
SELECT fecha, SUM(v.cantidad * p.precio_unitario * (1 - v.descuento_pct/100)) AS ingreso_bruto
FROM dbo.fact_ventas v
JOIN dbo.dim_productos p ON v.producto_id = p.producto_id
GROUP BY fecha
ORDER BY fecha;

SELECT AVG(x.ingreso_bruto) AS ingreso_diario_promedio
FROM (SELECT fecha, SUM(v.cantidad * p.precio_unitario * (1 - v.descuento_pct/100)) AS ingreso_bruto
      FROM dbo.fact_ventas v JOIN dbo.dim_productos p ON v.producto_id = p.producto_id
      GROUP BY fecha) x;

🟠 Ejercicio: Margen bruto total (ingreso - costo).

In [ ]:
SELECT SUM(v.cantidad * p.precio_unitario * (1 - v.descuento_pct/100)) - SUM(v.cantidad * p.costo_unitario) AS margen_total
FROM dbo.fact_ventas v
JOIN dbo.dim_productos p ON v.producto_id = p.producto_id;

🔴 Reto: Top 3 productos por ingreso acumulado.

In [ ]:
SELECT TOP 3 p.producto_id, p.nombre, SUM(v.cantidad * p.precio_unitario) AS ingreso
FROM dbo.fact_ventas v
JOIN dbo.dim_productos p ON v.producto_id = p.producto_id
GROUP BY p.producto_id, p.nombre
ORDER BY ingreso DESC;

Errores comunes: olvidar DISTINCT en conteos; no separar costos vs ingresos; mezclar granularidades.